# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [1]:
import pandas as pd
import numpy as np

# Load the actual recipe data
recipes = pd.read_csv('recipes.tsv', sep='\t')
recipe_tags = pd.read_csv('recipe-tags.tsv', sep='\t')

print("Recipe data loaded:")
print(f"Number of recipes: {len(recipes)}")
print(f"Number of recipe-tag pairs: {len(recipe_tags)}")

print("\nFirst few recipes:")
print(recipes.head())

print("\nFirst few recipe tags:")
print(recipe_tags.head())

print(f"\nAll recipe slugs ({len(recipes)} total):")
print(recipes['recipe_slug'].tolist())

Recipe data loaded:
Number of recipes: 100
Number of recipe-tag pairs: 752

First few recipes:
        recipe_slug      recipe_title  \
0           falafel           Falafel   
1        spamburger        Spamburger   
2  bacon-fried-rice  Bacon Fried Rice   
3   chicken-fingers   Chicken Fingers   
4       apple-crisp       Apple Crisp   

                                 recipe_introduction  
0  Falafel is a popular Middle Eastern dish made ...  
1  Spamburger is a type of hamburger that is made...  
2  Bacon fried rice is a savory and satisfying di...  
3  Chicken fingers are a popular dish made from c...  
4  Apple crisp is a classic dessert made with bak...  

First few recipe tags:
   recipe_slug recipe_tag
0  spam-musubi   hawaiian
1  spam-musubi       nori
2  spam-musubi    onthego
3  spam-musubi       rice
4  spam-musubi      snack

All recipe slugs (100 total):
['falafel', 'spamburger', 'bacon-fried-rice', 'chicken-fingers', 'apple-crisp', 'cranberry-apple-crisp', 'bacon-choco

In [ ]:
# Create ratings based on my very own love for baked goods and the fact I don't eat pork
# Raw ratings from 1-5, then convert to 0-1 scale
# Need at least 10 ratings of 1 (becomes 0.0) and 10 ratings of 5 (becomes 1.0)

# Create a function to get tags for a recipe
def get_recipe_tags(slug):
    return recipe_tags[recipe_tags['recipe_slug'] == slug]['recipe_tag'].tolist()

# Check which recipes contain pork/bacon
def contains_pork(slug):
    tags = get_recipe_tags(slug)
    pork_keywords = ['bacon', 'pork', 'ham', 'spam']
    return any(keyword in slug.lower() or keyword in str(tags).lower() for keyword in pork_keywords)

# Categorize recipes
pork_recipes = []
sweet_baked_goods = []
savory_dishes = []

for recipe in recipes['recipe_slug']:
    tags = get_recipe_tags(recipe)
    
    if contains_pork(recipe):
        pork_recipes.append(recipe)
    elif any(tag in ['dessert', 'baking', 'cake', 'cookies', 'brownies', 'pie', 'crisp', 'crumble', 'cobbler', 'chocolate', 'sweet', 'pastry'] for tag in tags):
        sweet_baked_goods.append(recipe)
    else:
        savory_dishes.append(recipe)

print(f"Pork/bacon recipes (will be rated low): {len(pork_recipes)}")
print(f"Sweet/baked goods (will be rated high): {len(sweet_baked_goods)}")
print(f"Savory dishes (mixed ratings): {len(savory_dishes)}")

# Define ratings for all recipes
ratings_raw = {}

# Rating 1 (becomes 0.0) - All pork dishes + some very savory/spicy dishes
disliked_recipes = pork_recipes + [
    'ma-la-chicken', 'ma-la-beef', 'chiles-rellenos', 'pickled-green-beans', 
    'pickled-asparagus', 'dan-dan-noodles', 'falafel', 'sujebi'
]

# Rating 2 (becomes 0.25) - Very savory dishes, not appealing to sweet tooth
neutral_low = [
    'spamburger', 'fried-oysters', 'guacamole', 'pico-de-gallo',
    'cold-noodles-with-tomatoes', 'cold-soba-noodles-with-tomato-and-cucumber',
    'cold-sesame-noodles', 'asparagus-burger', 'asparagus-quiche'
]

# Rating 3 (becomes 0.5) - Okay savory dishes
neutral_mid = [
    'chicken-fingers', 'chicken-nuggets', 'ramen', 'udon', 'tempura-udon',
    'tamales', 'enchiladas', 'tacos', 'quesadillas', 'burritos', 'fajitas',
    'nachos', 'loaded-nachos', 'nacho-dip', 'nacho-salad', 'nacho-casserole', 'nacho-fries',
    'taco-salad', 'pasta-primavera', 'vegetable-lasagna', 'spinach-quiche',
    'mushroom-quiche', 'spinach-and-feta-quiche', 'soba-noodle-salad-with-peanut-dressing'
]

# Rating 4 (becomes 0.75) - Good savory dishes + some breakfast items
liked_recipes = [
    'breakfast-burritos', 'bolognese-sauce', 'lasagna', 'chicken-alfredo-lasagna',
    'classic-beef-lasagna', 'vegetarian-mushroom-lasagna', 'spinach-and-ricotta-lasagna',
    'spinach-and-ricotta-stuffed-shells', 'french-toast', 'quiche-lorraine',
    'spam-musubi', 'souffle', 'vanilla-souffle', 'chocolate-souffle'
]

# Rating 5 (becomes 1.0) - All sweet/baked goods + some special breakfast pastries
loved_recipes = sweet_baked_goods + [
    'brioche-bread', 'brioche-bread-with-chocolate', 'croissants', 'butter-croissants',
    'chocolate-croissants', 'almond-croissants', 'croissant-aux-amandes', 'danishes'
]

# Assign ratings
for recipe in recipes['recipe_slug']:
    if recipe in disliked_recipes:
        ratings_raw[recipe] = 1
    elif recipe in neutral_low:
        ratings_raw[recipe] = 2  
    elif recipe in neutral_mid:
        ratings_raw[recipe] = 3
    elif recipe in liked_recipes:
        ratings_raw[recipe] = 4
    elif recipe in loved_recipes:
        ratings_raw[recipe] = 5
    else:
        # Default assignment for any missed recipes
        tags = get_recipe_tags(recipe)
        if contains_pork(recipe):
            ratings_raw[recipe] = 1
        elif any(tag in ['dessert', 'chocolate', 'sweet'] for tag in tags):
            ratings_raw[recipe] = 5
        else:
            ratings_raw[recipe] = 3

print(f"\nRating distribution (raw 1-5 scale):")
rating_counts = {}
for rating in [1, 2, 3, 4, 5]:
    count = sum(1 for r in ratings_raw.values() if r == rating)
    rating_counts[rating] = count
    print(f"Rating {rating}: {count} recipes")

print(f"\nTotal recipes rated: {len(ratings_raw)}")

# Check if we meet requirements
ratings_of_zero = rating_counts[1]  # Will become 0.0
ratings_of_one = rating_counts[5]   # Will become 1.0

print(f"\nRequirement check:")
print(f"Ratings that will become 0.0: {ratings_of_zero} (need >= 10)")
print(f"Ratings that will become 1.0: {ratings_of_one} (need >= 10)")
print(f"Requirements met: {ratings_of_zero >= 10 and ratings_of_one >= 10}")

# Show some examples of each category
print(f"\nExamples of lowest rated recipes (pork + very savory):")
lowest_rated = [recipe for recipe, rating in ratings_raw.items() if rating == 1][:10]
print(lowest_rated)

print(f"\nExamples of highest rated recipes (sweet/baked goods):")
highest_rated = [recipe for recipe, rating in ratings_raw.items() if rating == 5][:10]
print(highest_rated)

Pork/bacon recipes (will be rated low): 20
Sweet/baked goods (will be rated high): 33
Savory dishes (mixed ratings): 47

Rating distribution (raw 1-5 scale):
Rating 1: 27 recipes
Rating 2: 8 recipes
Rating 3: 23 recipes
Rating 4: 11 recipes
Rating 5: 31 recipes

Total recipes rated: 100

Requirement check:
Ratings that will become 0.0: 27 (need >= 10)
Ratings that will become 1.0: 31 (need >= 10)
Requirements met: True

Examples of lowest rated recipes (pork + very savory):
['falafel', 'spamburger', 'bacon-fried-rice', 'bacon-chocolate-chip-cookies', 'sujebi', 'ramen', 'bacon-wrapped-scallops', 'bacon-egg-muffins', 'breakfast-burritos', 'bacon-souffle']

Examples of highest rated recipes (sweet/baked goods):
['apple-crisp', 'cranberry-apple-crisp', 'brownies', 'croissants', 'butter-croissants', 'peanut-butter-brownies', 'almond-chip-cookies', 'brioche-bread', 'brioche-bread-with-chocolate', 'peanut-butter-cupcakes']


In [3]:
# Create the ratings DataFrame and convert to 0-1 scale
ratings_df = pd.DataFrame({
    'recipe_slug': list(ratings_raw.keys()),
    'rating_raw': list(ratings_raw.values())
})

# Convert to 0-1 scale as suggested in assignment
ratings_df['rating'] = (ratings_df['rating_raw'] - 1) * 0.25

# Sort by recipe slug for consistency
ratings_df = ratings_df.sort_values('recipe_slug')

print("Final ratings summary:")
print(f"Total recipes: {len(ratings_df)}")
print(f"Ratings of 0.0 (was rating 1): {sum(ratings_df['rating'] == 0.0)}")
print(f"Ratings of 0.25 (was rating 2): {sum(ratings_df['rating'] == 0.25)}")
print(f"Ratings of 0.5 (was rating 3): {sum(ratings_df['rating'] == 0.5)}")
print(f"Ratings of 0.75 (was rating 4): {sum(ratings_df['rating'] == 0.75)}")
print(f"Ratings of 1.0 (was rating 5): {sum(ratings_df['rating'] == 1.0)}")

print("\nFirst few ratings:")
print(ratings_df[['recipe_slug', 'rating']].head(10))

print("\nLast few ratings:")
print(ratings_df[['recipe_slug', 'rating']].tail(10))

# Save to TSV file
ratings_final = ratings_df[['recipe_slug', 'rating']]
ratings_final.to_csv('ratings.tsv', sep='\t', index=False)

print("\nRatings saved to 'ratings.tsv'")
print("File preview:")
print(ratings_final.head(15).to_csv(sep='\t', index=False))

Final ratings summary:
Total recipes: 100
Ratings of 0.0 (was rating 1): 27
Ratings of 0.25 (was rating 2): 8
Ratings of 0.5 (was rating 3): 23
Ratings of 0.75 (was rating 4): 11
Ratings of 1.0 (was rating 5): 31

First few ratings:
                         recipe_slug  rating
23               almond-chip-cookies    1.00
74                 almond-croissants    1.00
4                        apple-crisp    1.00
65                     apple-crumble    1.00
58                         apple-pie    1.00
28                  asparagus-burger    0.25
27                  asparagus-quiche    0.25
91  bacon-and-egg-breakfast-sandwich    0.00
6       bacon-chocolate-chip-cookies    0.00
12                 bacon-egg-muffins    0.00

Last few ratings:
                    recipe_slug  rating
61     strawberry-rhubarb-crisp    1.00
7                        sujebi    0.00
72                   taco-salad    0.50
33                        tacos    0.50
31                      tamales    0.50
56           

Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [5]:
# YOUR CHANGES HERE

# Load the ratings to ensure same order
ratings = pd.read_csv('ratings.tsv', sep='\t')
recipe_tags = pd.read_csv('recipe-tags.tsv', sep='\t')

print(f"Ratings loaded: {len(ratings)} recipes")
print(f"Recipe tags loaded: {len(recipe_tags)} tag entries")

# Get all unique tags
all_tags = sorted(recipe_tags['recipe_tag'].unique())
print(f"Number of unique tags: {len(all_tags)}")
print(f"First 10 tags: {all_tags[:10]}")

# Create one-hot encoding for tags using pandas pivot
# - creating a matrix where each row is a recipe and each column is a tag (0/1)
tag_matrix = recipe_tags.pivot_table(
    index='recipe_slug', 
    columns='recipe_tag', 
    aggfunc=len,  # Count occurrences (should be 1 for each recipe-tag pair)
    fill_value=0  # Fill missing combinations with 0
)

# Convert to binary (in case there are any values > 1)
tag_matrix = (tag_matrix > 0).astype(int)

print(f"\nTag matrix shape: {tag_matrix.shape}")
print(f"Recipes in tag matrix: {len(tag_matrix)}")

# Reset index to make recipe_slug a column
tag_features = tag_matrix.reset_index()

print(f"\nTag features shape after reset: {tag_features.shape}")
print(f"Columns: {list(tag_features.columns)[:10]}...")

# Add bias column (hardcoded to 1 as specified)
tag_features.insert(1, 'bias', 1)

print(f"\nAfter adding bias column: {tag_features.shape}")
print(f"First few columns: {list(tag_features.columns)[:5]}")

# Ensure we have all recipes from ratings and in the same order
print(f"\nRecipes in ratings: {len(ratings)}")
print(f"Recipes in tag_features: {len(tag_features)}")

# Check if all recipes from ratings are in tag_features
missing_recipes = set(ratings['recipe_slug']) - set(tag_features['recipe_slug'])
extra_recipes = set(tag_features['recipe_slug']) - set(ratings['recipe_slug'])

print(f"Missing recipes in tag_features: {len(missing_recipes)}")
if missing_recipes:
    print(f"Missing: {missing_recipes}")

print(f"Extra recipes in tag_features: {len(extra_recipes)}")
if extra_recipes:
    print(f"Extra: {extra_recipes}")

# Merge with ratings to ensure same order and completeness
features = ratings[['recipe_slug']].merge(tag_features, on='recipe_slug', how='left')

# Fill any missing tag values with 0 (for recipes that don't appear in recipe-tags.tsv)
features = features.fillna(0)

print(f"\nFinal features shape: {features.shape}")
print(f"Features columns: bias + {len(all_tags)} tags = {1 + len(all_tags)} total")

# Display first few rows and columns
print(f"\nFirst 5 recipes and first 10 features:")
print(features.iloc[:5, :11])

print(f"\nLast 5 recipes:")
print(features[['recipe_slug']].tail())

Ratings loaded: 100 recipes
Recipe tags loaded: 752 tag entries
Number of unique tags: 296
First 10 tags: ['alfredo', 'almond', 'american', 'appetizer', 'appetizers', 'apple', 'asiancuisine', 'asparagus', 'avocado', 'bacon']

Tag matrix shape: (100, 296)
Recipes in tag matrix: 100

Tag features shape after reset: (100, 297)
Columns: ['recipe_slug', 'alfredo', 'almond', 'american', 'appetizer', 'appetizers', 'apple', 'asiancuisine', 'asparagus', 'avocado']...

After adding bias column: (100, 298)
First few columns: ['recipe_slug', 'bias', 'alfredo', 'almond', 'american']

Recipes in ratings: 100
Recipes in tag_features: 100
Missing recipes in tag_features: 0
Extra recipes in tag_features: 0

Final features shape: (100, 298)
Features columns: bias + 296 tags = 297 total

First 5 recipes and first 10 features:
           recipe_slug  bias  alfredo  almond  american  appetizer  \
0  almond-chip-cookies     1        0       1         0          0   
1    almond-croissants     1        0    

In [7]:
# Save features to TSV file
features.to_csv('features.tsv', sep='\t', index=False)
print("Features saved to 'features.tsv'")

# Verify alignment between ratings and features
ratings_check = pd.read_csv('ratings.tsv', sep='\t')
features_check = pd.read_csv('features.tsv', sep='\t')

print(f"\nVerification:")
print(f"Ratings shape: {ratings_check.shape}")
print(f"Features shape: {features_check.shape}")

# Check if recipe order is identical
recipe_order_match = (ratings_check['recipe_slug'] == features_check['recipe_slug']).all()
print(f"Recipe order matches: {recipe_order_match}")

if recipe_order_match:
    print("Files are properly aligned and everything works")
else:
    print("Files are NOT aligned - this needs to be fixed - FAIL")

# Show summary statistics
print(f"\nFeature matrix summary:")
print(f"- Total recipes: {len(features)}")
print(f"- Total features: {features.shape[1] - 1} (excluding recipe_slug)")  
print(f"- Bias column: always 1")
print(f"- Tag features: {features.shape[1] - 2} binary features")

# Show some examples of feature vectors
print(f"\nExample feature vectors (showing first 10 features):")
# First 5 recipes, first 11 columns
example_features = features.iloc[:5, :11]  
print(example_features)

# Check sparsity of the feature matrix
# Exclude recipe_slug and bias
feature_matrix = features.iloc[:, 2:].values  
total_elements = feature_matrix.size
nonzero_elements = np.count_nonzero(feature_matrix)
sparsity = 1 - (nonzero_elements / total_elements)

print(f"\nFeature matrix sparsity:")
print(f"- Total elements: {total_elements}")
print(f"- Non-zero elements: {nonzero_elements}")
print(f"- Sparsity: {sparsity:.3f} ({sparsity*100:.1f}% zeros)")

# Show tag distribution
# Count how many recipes have each tag
tag_counts = feature_matrix.sum(axis=0) 
print(f"\nTag usage statistics:")
print(f"- Most common tags (top 10):")
tag_names = features.columns[2:].tolist()
tag_freq = list(zip(tag_names, tag_counts))
tag_freq_sorted = sorted(tag_freq, key=lambda x: x[1], reverse=True)
for tag, count in tag_freq_sorted[:10]:
    print(f"  {tag}: {count} recipes")

print(f"- Least common tags (bottom 10):")
for tag, count in tag_freq_sorted[-10:]:
    print(f"  {tag}: {count} recipes")

Features saved to 'features.tsv'

Verification:
Ratings shape: (100, 2)
Features shape: (100, 298)
Recipe order matches: True
Files are properly aligned and everything works

Feature matrix summary:
- Total recipes: 100
- Total features: 297 (excluding recipe_slug)
- Bias column: always 1
- Tag features: 296 binary features

Example feature vectors (showing first 10 features):
           recipe_slug  bias  alfredo  almond  american  appetizer  \
0  almond-chip-cookies     1        0       1         0          0   
1    almond-croissants     1        0       1         0          0   
2          apple-crisp     1        0       0         0          0   
3        apple-crumble     1        0       0         0          0   
4            apple-pie     1        0       0         0          0   

   appetizers  apple  asiancuisine  asparagus  avocado  
0           0      0             0          0        0  
1           0      0             0          0        0  
2           0      1        

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [9]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

# Load the aligned data
features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

# Verify alignment
assert (features['recipe_slug'] == ratings['recipe_slug']).all(), "Data not aligned!"

print("Data loaded and verified:")
print(f"Features shape: {features.shape}")
print(f"Ratings shape: {ratings.shape}")

# Prepare feature matrix (exclude recipe_slug column)
X = features.drop('recipe_slug', axis=1).values
y = ratings['rating'].values

print(f"\nFeature matrix X shape: {X.shape}")
print(f"Target vector y shape: {y.shape}")
print(f"y range: {y.min()} to {y.max()}")

# Build ridge regression model with alpha=1 and fit_intercept=False
# fit_intercept=False because we already have a bias column
ridge_model = Ridge(alpha=1.0, fit_intercept=False, random_state=42)
ridge_model.fit(X, y)

print(f"\nRidge regression model trained:")
print(f"Number of coefficients: {len(ridge_model.coef_)}")
print(f"Expected number: {X.shape[1]} (bias + {X.shape[1]-1} tags)")

# Model performance metrics
y_pred = ridge_model.predict(X)
mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print(f"\nModel Performance:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R² Score: {r2:.4f}")

# Show coefficient statistics
coefficients = ridge_model.coef_
print(f"\nCoefficient Statistics:")
print(f"Min coefficient: {coefficients.min():.4f}")
print(f"Max coefficient: {coefficients.max():.4f}")
print(f"Mean absolute coefficient: {np.abs(coefficients).mean():.4f}")

# Show bias coefficient (first coefficient)
print(f"Bias coefficient: {coefficients[0]:.4f}")

# Show some example predictions vs actual ratings
print(f"\nExample Predictions (first 10 recipes):")
for i in range(10):
    recipe = features.iloc[i]['recipe_slug']
    actual = y[i]
    predicted = y_pred[i]
    print(f"{recipe:25} | Actual: {actual:.2f} | Predicted: {predicted:.2f} | Error: {actual-predicted:.3f}")

# Show top positive and negative tag coefficients (excluding bias)
feature_names = features.columns[1:].tolist()  # Exclude recipe_slug
coef_with_names = list(zip(feature_names, coefficients))

# Sort by coefficient value
coef_sorted = sorted(coef_with_names[1:], key=lambda x: x[1], reverse=True)  # Exclude bias

print(f"\nTop 10 most positive tag coefficients (preference indicators):")
for tag, coef in coef_sorted[:10]:
    print(f"  {tag:20}: {coef:+.4f}")

print(f"\nTop 10 most negative tag coefficients (dislike indicators):")
for tag, coef in coef_sorted[-10:]:
    print(f"  {tag:20}: {coef:+.4f}")

Data loaded and verified:
Features shape: (100, 298)
Ratings shape: (100, 2)

Feature matrix X shape: (100, 297)
Target vector y shape: (100,)
y range: 0.0 to 1.0

Ridge regression model trained:
Number of coefficients: 297
Expected number: 297 (bias + 296 tags)

Model Performance:
Mean Squared Error: 0.0019
R² Score: 0.9876

Coefficient Statistics:
Min coefficient: -0.3440
Max coefficient: 0.3526
Mean absolute coefficient: 0.0403
Bias coefficient: 0.3526

Example Predictions (first 10 recipes):
almond-chip-cookies       | Actual: 1.00 | Predicted: 0.94 | Error: 0.063
almond-croissants         | Actual: 1.00 | Predicted: 1.03 | Error: -0.025
apple-crisp               | Actual: 1.00 | Predicted: 0.99 | Error: 0.005
apple-crumble             | Actual: 1.00 | Predicted: 1.02 | Error: -0.018
apple-pie                 | Actual: 1.00 | Predicted: 1.00 | Error: -0.001
asparagus-burger          | Actual: 0.25 | Predicted: 0.25 | Error: 0.001
asparagus-quiche          | Actual: 0.25 | Predicted

Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [10]:
# YOUR CHANGES HERE

# Create model coefficients DataFrame
# feature_names includes 'bias' + all tag names (excluding recipe_slug)
feature_names = features.columns[1:].tolist()  # Skip recipe_slug column
coefficients = ridge_model.coef_

print(f"Creating model coefficients file:")
print(f"Number of features: {len(feature_names)}")
print(f"Number of coefficients: {len(coefficients)}")

# Create DataFrame with recipe_tag and coefficient columns
model_df = pd.DataFrame({
    'recipe_tag': feature_names,
    'coefficient': coefficients
})

print(f"\nModel coefficients summary:")
print(f"Shape: {model_df.shape}")
print(f"First few entries:")
print(model_df.head())

print(f"\nLast few entries:")
print(model_df.tail())

# Save to TSV file
model_df.to_csv('model.tsv', sep='\t', index=False)
print(f"\nModel coefficients saved to 'model.tsv'")

# Verification - load and check
model_check = pd.read_csv('model.tsv', sep='\t')
print(f"Verification - loaded model.tsv:")
print(f"Shape: {model_check.shape}")
print(f"Columns: {model_check.columns.tolist()}")

# Show most important coefficients again for verification
print(f"\nTop 5 most positive coefficients (loves these):")
top_positive = model_df.nlargest(5, 'coefficient')
for _, row in top_positive.iterrows():
    print(f"  {row['recipe_tag']:20}: {row['coefficient']:+.4f}")

print(f"\nTop 5 most negative coefficients (dislikes these):")
top_negative = model_df.nsmallest(5, 'coefficient')
for _, row in top_negative.iterrows():
    print(f"  {row['recipe_tag']:20}: {row['coefficient']:+.4f}")

# Check that bias is included
bias_coef = model_df[model_df['recipe_tag'] == 'bias']['coefficient'].iloc[0]
print(f"\nBias coefficient: {bias_coef:.4f}")
print(f"This represents the baseline preference level before considering any specific tags.")

Creating model coefficients file:
Number of features: 297
Number of coefficients: 297

Model coefficients summary:
Shape: (297, 2)
First few entries:
  recipe_tag  coefficient
0       bias     0.352603
1    alfredo     0.047706
2     almond     0.105088
3   american     0.032247
4  appetizer    -0.043827

Last few entries:
       recipe_tag  coefficient
292    vegetarian    -0.014240
293          warm     0.035646
294  whippedcream    -0.017843
295        winter     0.031184
296    yeastdough     0.022446

Model coefficients saved to 'model.tsv'
Verification - loaded model.tsv:
Shape: (297, 2)
Columns: ['recipe_tag', 'coefficient']

Top 5 most positive coefficients (loves these):
  bias                : +0.3526
  baking              : +0.2521
  dessert             : +0.2051
  french              : +0.1595
  frenchpastry        : +0.1580

Top 5 most negative coefficients (dislikes these):
  bacon               : -0.3440
  creamcheese         : -0.1527
  danish              : -0.1527
  h

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [11]:
# YOUR CHANGES HERE

# Load the feature data and use our trained model to generate estimates
features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

# Prepare feature matrix (exclude recipe_slug column)
X = features.drop('recipe_slug', axis=1).values

# Generate score estimates using the trained ridge regression model
score_estimates = ridge_model.predict(X)

print(f"Generated score estimates:")
print(f"Number of recipes: {len(score_estimates)}")
print(f"Estimate range: {score_estimates.min():.4f} to {score_estimates.max():.4f}")

# Create estimates DataFrame
estimates_df = pd.DataFrame({
    'recipe_slug': features['recipe_slug'],
    'score_estimate': score_estimates
})

print(f"\nEstimates DataFrame shape: {estimates_df.shape}")
print(f"First few estimates:")
print(estimates_df.head(10))

# Compare estimates with actual ratings for verification
comparison = estimates_df.merge(ratings, on='recipe_slug')
comparison['error'] = comparison['rating'] - comparison['score_estimate']
comparison['abs_error'] = abs(comparison['error'])

print(f"\nComparison with actual ratings:")
print(f"Mean Absolute Error: {comparison['abs_error'].mean():.4f}")
print(f"Max Absolute Error: {comparison['abs_error'].max():.4f}")

# Show recipes with highest estimated scores (model's top recommendations)
print(f"\nTop 10 highest estimated scores (model recommendations):")
top_estimates = estimates_df.nlargest(10, 'score_estimate')
for _, row in top_estimates.iterrows():
    actual_rating = ratings[ratings['recipe_slug'] == row['recipe_slug']]['rating'].iloc[0]
    print(f"{row['recipe_slug']:25} | Estimate: {row['score_estimate']:.3f} | Actual: {actual_rating:.3f}")

# Show recipes with lowest estimated scores
print(f"\nTop 10 lowest estimated scores:")
low_estimates = estimates_df.nsmallest(10, 'score_estimate')
for _, row in low_estimates.iterrows():
    actual_rating = ratings[ratings['recipe_slug'] == row['recipe_slug']]['rating'].iloc[0]
    print(f"{row['recipe_slug']:25} | Estimate: {row['score_estimate']:.3f} | Actual: {actual_rating:.3f}")

# Save estimates to TSV file
estimates_df.to_csv('estimates.tsv', sep='\t', index=False)
print(f"\nScore estimates saved to 'estimates.tsv'")

# Verification
estimates_check = pd.read_csv('estimates.tsv', sep='\t')
print(f"Verification - loaded estimates.tsv:")
print(f"Shape: {estimates_check.shape}")
print(f"Columns: {estimates_check.columns.tolist()}")
print(f"Sample entries:")
print(estimates_check.head())

Generated score estimates:
Number of recipes: 100
Estimate range: -0.0178 to 1.0498

Estimates DataFrame shape: (100, 2)
First few estimates:
                        recipe_slug  score_estimate
0               almond-chip-cookies        0.936994
1                 almond-croissants        1.025366
2                       apple-crisp        0.994851
3                     apple-crumble        1.017843
4                         apple-pie        1.001116
5                  asparagus-burger        0.248667
6                  asparagus-quiche        0.265099
7  bacon-and-egg-breakfast-sandwich        0.001166
8      bacon-chocolate-chip-cookies        0.103821
9                 bacon-egg-muffins        0.025735

Comparison with actual ratings:
Mean Absolute Error: 0.0339
Max Absolute Error: 0.1527

Top 10 highest estimated scores (model recommendations):
blueberry-crisp           | Estimate: 1.050 | Actual: 1.000
brioche-bread-with-chocolate | Estimate: 1.048 | Actual: 1.000
almond-croissants

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [12]:
# YOUR CHANGES HERE

# Load data
features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')
estimates = pd.read_csv('estimates.tsv', sep='\t')

# Prepare data matrices
X = features.drop('recipe_slug', axis=1).values  # Feature matrix (100 x 297)
y = ratings['rating'].values  # Ratings vector (100,)
recipe_slugs = features['recipe_slug'].values

print(f"Data loaded:")
print(f"Feature matrix X shape: {X.shape}")
print(f"Ratings vector y shape: {y.shape}")
print(f"Number of recipes: {len(recipe_slugs)}")

# LinUCB parameters
ridge_alpha = 1.0  # Regularization parameter (same as Part 3)
linucb_alpha = 2.0  # LinUCB confidence parameter

print(f"\nLinUCB Parameters:")
print(f"Ridge regularization (alpha): {ridge_alpha}")
print(f"LinUCB confidence (alpha): {linucb_alpha}")

# Simulate LinUCB scenario: we've tried every recipe once
# This means our design matrix D is just X (all features)
# and our reward vector c is just y (all ratings)

# Calculate A = D^T D + I_d (where D is our feature matrix X)
# A represents the precision matrix for the posterior
d = X.shape[1]  # Number of features (297)
I_d = np.eye(d)  # Identity matrix
A = X.T @ X + ridge_alpha * I_d  # (297 x 297)

print(f"Matrix A shape: {A.shape}")
print(f"A condition number: {np.linalg.cond(A):.2e}")

# Calculate A_inv for confidence bound computation
A_inv = np.linalg.inv(A)
print(f"A_inv computed successfully")

# Calculate b = D^T c (where c is reward vector y)
b = X.T @ y  # (297,)

# Calculate theta_hat = A_inv @ b (ridge regression coefficients)
theta_hat = A_inv @ b

print(f"Theta_hat shape: {theta_hat.shape}")
print(f"Theta_hat range: {theta_hat.min():.4f} to {theta_hat.max():.4f}")

# Verify our theta_hat matches the ridge regression from Part 3
ridge_model = Ridge(alpha=ridge_alpha, fit_intercept=False)
ridge_model.fit(X, y)
ridge_coef = ridge_model.coef_

print(f"\nCoefficient verification:")
print(f"Max difference with Ridge: {np.max(np.abs(theta_hat - ridge_coef)):.6f}")
print(f"Coefficients match: {np.allclose(theta_hat, ridge_coef)}")

# Calculate LinUCB upper bounds for each recipe
# Upper bound = z^T theta_hat + alpha * sqrt(z^T A_inv z)
# where z is the feature vector for each recipe

score_bounds = np.zeros(len(recipe_slugs))
score_estimates_check = np.zeros(len(recipe_slugs))

for i in range(len(recipe_slugs)):
    z = X[i, :]  # Feature vector for recipe i
    
    # Point estimate: z^T theta_hat
    point_estimate = z @ theta_hat
    score_estimates_check[i] = point_estimate
    
    # Confidence width: alpha * sqrt(z^T A_inv z)
    confidence_width = linucb_alpha * np.sqrt(z @ A_inv @ z)
    
    # Upper confidence bound
    upper_bound = point_estimate + confidence_width
    score_bounds[i] = upper_bound

print(f"\nLinUCB bounds calculated:")
print(f"Score bounds range: {score_bounds.min():.4f} to {score_bounds.max():.4f}")
print(f"Point estimates range: {score_estimates_check.min():.4f} to {score_estimates_check.max():.4f}")

# Verify point estimates match our previous estimates
estimates_diff = np.max(np.abs(score_estimates_check - estimates['score_estimate'].values))
print(f"Max difference with previous estimates: {estimates_diff:.6f}")

# Create bounds DataFrame
bounds_df = pd.DataFrame({
    'recipe_slug': recipe_slugs,
    'score_bound': score_bounds
})

print(f"\nBounds DataFrame shape: {bounds_df.shape}")
print(f"First few bounds:")
print(bounds_df.head(10))

# Show recipes with highest upper bounds (LinUCB recommendations)
print(f"\nTop 10 highest LinUCB upper bounds:")
top_bounds = bounds_df.nlargest(10, 'score_bound')
for _, row in top_bounds.iterrows():
    recipe = row['recipe_slug']
    bound = row['score_bound']
    estimate = estimates[estimates['recipe_slug'] == recipe]['score_estimate'].iloc[0]
    actual = ratings[ratings['recipe_slug'] == recipe]['rating'].iloc[0]
    confidence_width = bound - estimate
    print(f"{recipe:25} | Bound: {bound:.3f} | Est: {estimate:.3f} | Actual: {actual:.3f} | Width: {confidence_width:.3f}")

# Save bounds to TSV file
bounds_df.to_csv('bounds.tsv', sep='\t', index=False)
print(f"\nLinUCB bounds saved to 'bounds.tsv'")

# Verification
bounds_check = pd.read_csv('bounds.tsv', sep='\t')
print(f"Verification - loaded bounds.tsv:")
print(f"Shape: {bounds_check.shape}")
print(f"Columns: {bounds_check.columns.tolist()}")

Data loaded:
Feature matrix X shape: (100, 297)
Ratings vector y shape: (100,)
Number of recipes: 100

LinUCB Parameters:
Ridge regularization (alpha): 1.0
LinUCB confidence (alpha): 2.0
Matrix A shape: (297, 297)
A condition number: 1.55e+02
A_inv computed successfully
Theta_hat shape: (297,)
Theta_hat range: -0.3440 to 0.3526

Coefficient verification:
Max difference with Ridge: 0.000000
Coefficients match: True

LinUCB bounds calculated:
Score bounds range: 1.5984 to 2.8875
Point estimates range: -0.0178 to 1.0498
Max difference with previous estimates: 0.000000

Bounds DataFrame shape: (100, 2)
First few bounds:
                        recipe_slug  score_bound
0               almond-chip-cookies     2.660453
1                 almond-croissants     2.731221
2                       apple-crisp     2.771193
3                     apple-crumble     2.887515
4                         apple-pie     2.851369
5                  asparagus-burger     2.112670
6                  asparagus-quic

Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [15]:
import matplotlib.pyplot as plt

# Load data again just to be safe
features = pd.read_csv('features.tsv', sep='\t')
ratings = pd.read_csv('ratings.tsv', sep='\t')

# Prepare data
X = features.drop('recipe_slug', axis=1).values  # Feature matrix (100 x 297)
recipe_slugs = features['recipe_slug'].values
recipe_ratings = dict(zip(ratings['recipe_slug'], ratings['rating']))

print(f"Online LinUCB Setup:")
print(f"Number of recipes: {len(recipe_slugs)}")
print(f"Feature dimensions: {X.shape[1]}")

# LinUCB parameters
ridge_alpha = 1.0
linucb_alpha = 2.0
d = X.shape[1]

print(f"Parameters: ridge_alpha={ridge_alpha}, linucb_alpha={linucb_alpha}")

# Initialize LinUCB state
A = ridge_alpha * np.eye(d)  # Start with regularization matrix
b = np.zeros(d)  # Start with zero reward accumulation
recommendations = []  # Track recommendations made

# Create recipe index mapping for quick lookup
recipe_to_idx = {slug: i for i, slug in enumerate(recipe_slugs)}

# Track which recipes have been recommended
recommended_recipes = set()

# Make 100 recommendations (one for each recipe)
for round_num in range(100):
    if round_num % 10 == 0:
        print(f"Round {round_num + 1}/100")
    
    # Calculate current parameter estimate
    A_inv = np.linalg.inv(A)
    theta_hat = A_inv @ b
    
    # Calculate upper confidence bounds for all remaining recipes
    best_recipe = None
    best_bound = -np.inf
    best_idx = -1
    
    for i, recipe in enumerate(recipe_slugs):
        # Skip if already recommended
        if recipe in recommended_recipes:
            continue
            
        z = X[i, :]  # Feature vector for this recipe
        
        # Calculate upper confidence bound
        point_estimate = z @ theta_hat
        confidence_width = linucb_alpha * np.sqrt(z @ A_inv @ z)
        upper_bound = point_estimate + confidence_width
        
        # Track best (break ties arbitrarily - first encountered wins)
        if upper_bound > best_bound:
            best_bound = upper_bound
            best_recipe = recipe
            best_idx = i
    
    # Ensure we found a recipe
    if best_recipe is None:
        print(f"Error: No unrecommended recipes found at round {round_num + 1}")
        break
    
    # Make recommendation
    recommended_recipe = best_recipe
    score_bound = best_bound
    reward = recipe_ratings[recommended_recipe]
    
    # Record recommendation
    recommendations.append({
        'recipe_slug': recommended_recipe,
        'score_bound': score_bound,
        'reward': reward
    })
    
    # Mark as recommended
    recommended_recipes.add(recommended_recipe)
    
    # Update LinUCB state with observed reward
    z = X[best_idx, :]  # Feature vector of chosen recipe
    A += np.outer(z, z)  # A = A + z * z^T
    b += reward * z      # b = b + reward * z

print(f"Completed {len(recommendations)} recommendations")

# Create recommendations DataFrame
recommendations_df = pd.DataFrame(recommendations)

print(f"\nRecommendations summary:")
print(f"Shape: {recommendations_df.shape}")
print(f"Average reward: {recommendations_df['reward'].mean():.4f}")
print(f"Total reward: {recommendations_df['reward'].sum():.2f}")

print(f"\nFirst 10 recommendations:")
for i in range(10):
    rec = recommendations[i]
    print(f"{i+1:2d}: {rec['recipe_slug']:25} | Bound: {rec['score_bound']:.3f} | Reward: {rec['reward']:.2f}")

print(f"\nLast 10 recommendations:")
for i in range(90, 100):
    rec = recommendations[i]
    print(f"{i+1:2d}: {rec['recipe_slug']:25} | Bound: {rec['score_bound']:.3f} | Reward: {rec['reward']:.2f}")

# Analyze performance over time
cumulative_reward = np.cumsum(recommendations_df['reward'].values)
cumulative_regret = np.arange(1, len(recommendations) + 1) - cumulative_reward  # Assume max possible reward is 1

print(f"\nPerformance analysis:")
print(f"Final cumulative reward: {cumulative_reward[-1]:.2f}")
print(f"Final cumulative regret: {cumulative_regret[-1]:.2f}")
print(f"Average reward per recommendation: {cumulative_reward[-1] / len(recommendations):.4f}")

# Count rewards by rating level
reward_counts = recommendations_df['reward'].value_counts().sort_index()
print(f"\nReward distribution:")
for reward, count in reward_counts.items():
    print(f"  Reward {reward}: {count} recipes")

# Save recommendations to file
recommendations_df.to_csv('recommendations.tsv', sep='\t', index=False)
print(f"\nRecommendations saved to 'recommendations.tsv'")

# Verification
recommendations_check = pd.read_csv('recommendations.tsv', sep='\t')
print(f"Verification - loaded recommendations.tsv:")
print(f"Shape: {recommendations_check.shape}")
print(f"Columns: {recommendations_check.columns.tolist()}")
print(f"Sample entries:")
print(recommendations_check.head())

Online LinUCB Setup:
Number of recipes: 100
Feature dimensions: 297
Parameters: ridge_alpha=1.0, linucb_alpha=2.0
Round 1/100
Round 11/100
Round 21/100
Round 31/100
Round 41/100
Round 51/100
Round 61/100
Round 71/100
Round 81/100
Round 91/100
Completed 100 recommendations

Recommendations summary:
Shape: (100, 3)
Average reward: 0.5275
Total reward: 52.75

First 10 recommendations:
 1: apple-crumble             | Bound: 7.483 | Reward: 1.00
 2: ramen                     | Bound: 7.270 | Reward: 0.00
 3: quesadillas               | Bound: 7.236 | Reward: 0.50
 4: ma-la-chicken             | Bound: 7.161 | Reward: 0.00
 5: chocolate-babka           | Bound: 6.963 | Reward: 1.00
 6: pain-au-chocolat          | Bound: 7.011 | Reward: 1.00
 7: spamburger                | Bound: 6.988 | Reward: 0.00
 8: bacon-fried-rice          | Bound: 6.883 | Reward: 0.00
 9: nacho-fries               | Bound: 6.690 | Reward: 0.50
10: cranberry-relish          | Bound: 6.692 | Reward: 1.00

Last 10 recomm

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.